In [1]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.io as pio
import pandas as pd
from scipy.stats import gmean
from IPython.display import IFrame

## Importance of the order and of the geometry preservation

In [2]:
dataset = pd.read_csv('CollectedData.csv')
dataset = dataset.drop({"Unnamed: 0"},axis=1)
dataset["Order"] = (dataset["Integrator"]<2).astype(np.compat.long)
importanceOrder = dataset.groupby(["Order","Integrator"],as_index=False)["E1"].apply(gmean).sort_values("Order")
importanceOrder["E2"] = dataset.groupby(["Order","Integrator"],as_index=False)["E2"].apply(gmean).sort_values("Order")["E2"]
importanceOrder["Training Loss"] = dataset.groupby(["Order","Integrator"],as_index=False)["Training Loss"].apply(gmean).sort_values("Order")["Training Loss"]
importanceOrder["Integrator"] = importanceOrder["Integrator"].astype(np.compat.long)
listIntegrators = ["RK4", "CF4", "EE", "LE"]
listOrders = [1,4]
importanceOrder["Integrator"] = [listIntegrators[i] for i in  importanceOrder["Integrator"]]
importanceOrder["Order"] = [listOrders[i] for i in  importanceOrder["Order"]]
importanceOrder

,Order,Integrator,E1,E2,Training Loss
0,1,EE,0.000057,0.011321,2.118858e-06
1,1,LE,0.000049,0.010694,1.169988e-06
2,4,RK4,0.000012,0.003829,2.626100e-07
3,4,CF4,0.000012,0.003845,2.641869e-07


## Plot with all the 5 experiments for each configuration

In [3]:
dataset = pd.read_csv('CollectedData.csv')
dataset = dataset.drop("Unnamed: 0",axis=1)
dataset["E1"] = np.log10(dataset["E1"])
dataset["E2"] = np.log10(dataset["E2"])
dataset["Training Loss"] = np.log10(dataset["Training Loss"])

In [4]:
dataset['Epsilon'][dataset['Epsilon']==0] = 1e-4 #just for the plot
dataset["Integrator"] = dataset["Integrator"].astype(np.compat.long)
dataset['Epsilon'] = np.log10(dataset['Epsilon'])
threshold = 1e-7
dataset["Threshold"] = (dataset["E1"]<np.log10(threshold)).astype(np.compat.long)

In [5]:
import plotly.graph_objects as go

vecE1 = [np.round(min(dataset["E1"]),2),*np.round(np.linspace(min(dataset["E1"])+1, max(dataset["E1"])-1, 4),0), np.round(max(dataset["E1"]),2)]
vecE2 = [np.round(min(dataset["E2"]),2),*np.round(np.linspace(min(dataset["E2"])+1, max(dataset["E2"])-1, 3),0), np.round(max(dataset["E2"]),2)]
vecLoss = [np.round(min(dataset["Training Loss"]),2),*np.round(np.linspace(min(dataset["Training Loss"])+1, max(dataset["Training Loss"])-1, 4),0), np.round(max(dataset["Training Loss"]),2)]


fig = go.Figure(data=go.Parcoords(line = dict(color=dataset["Threshold"], colorscale=[[0,"pink"],[1,"cyan"]]),
                              dimensions = list([
                              dict(label = "N", values = dataset['N'], tickvals = [50,500,1000,1500]),
                              dict(label = 'M', values = dataset['M'], tickvals = [2,3,5]),
                              dict(label = 'Epsilon', values = dataset["Epsilon"],
                                   tickvals = [-4,-3,-2,-1],
                                   ticktext = ["0","0.001","0.01","0.1"]),
                              dict(label = "Integrator", values = dataset["Integrator"],
                                   tickvals = [0,1,2,3],
                                   ticktext = ["RK4", "CF4", "EE", "LE"]),
                              dict(label = 'log10(E1)', values = (dataset['E1']), tickvals = vecE1 ),
                              dict(label = 'log10(E2)', values = (dataset['E2']), tickvals = vecE2),
                              dict(label = 'log10(Loss)', values = (dataset['Training Loss']), tickvals = vecLoss)
                          ])
                          )
                              )
fig = fig.update_layout(
    title=dict(text = "Results with the repeated experiments", x=.5, xanchor="center", yanchor= "top")
)
fig = fig.update_traces(tickfont_size=15, labelfont_size = 15, 
                    tickfont_color = "black", selector=dict(type='parcoords'),
                    labelside = "bottom")
#fig.show() # this is the plot downloaded as html and displayed as interactive plot below
from IPython.display import IFrame
url = "https://htmlpreview.github.io/?https://github.com/davidemurari/learningConstrainedHamiltonians/blob/main/Constrained%20Systems/AllExperiments.html"
IFrame(src=url, width=1000, height=500)

## Plot based on the median over the 5 experiments

In [6]:
dataset = pd.read_csv('CollectedData.csv')
dataset = dataset.drop("Unnamed: 0",axis=1)

dataset = dataset.groupby(['N',"M","Integrator","Epsilon"],as_index=False)[["E1","E2","Training Loss"]].median()

In [7]:
dataset["E1"] = np.log10(dataset["E1"])
dataset["E2"] = np.log10(dataset["E2"])
dataset["Training Loss"] = np.log10(dataset["Training Loss"])

In [8]:
dataset['Epsilon'][dataset['Epsilon']==0] = 1e-4 #just for the plot
dataset["Integrator"] = dataset["Integrator"].astype(np.compat.long)
dataset['Epsilon'] = np.log10(dataset['Epsilon'])
threshold = 1e-7
dataset["Threshold"] = (dataset["E1"]<np.log10(threshold)).astype(np.compat.long)

In [9]:
import plotly.graph_objects as go

vecE1 = [np.round(min(dataset["E1"]),2),*np.round(np.linspace(min(dataset["E1"])+1, max(dataset["E1"])-1, 4),0), np.round(max(dataset["E1"]),2)]
vecE2 = [np.round(min(dataset["E2"]),2),*np.round(np.linspace(min(dataset["E2"])+1, max(dataset["E2"])-1, 3),0), np.round(max(dataset["E2"]),2)]
vecLoss = [np.round(min(dataset["Training Loss"]),2),*np.round(np.linspace(min(dataset["Training Loss"])+1, max(dataset["Training Loss"])-1, 4),0), np.round(max(dataset["Training Loss"]),2)]


fig = go.Figure(data=go.Parcoords(line = dict(color=dataset["Threshold"], colorscale=[[0,"pink"],[1,"cyan"]]),
                              dimensions = list([
                              dict(label = "N", values = dataset['N'], tickvals = [50,500,1000,1500]),
                              dict(label = 'M', values = dataset['M'], tickvals = [2,3,5]),
                              dict(label = 'Epsilon', values = dataset["Epsilon"],
                                   tickvals = [-4,-3,-2,-1],
                                   ticktext = ["0","0.001","0.01","0.1"]),
                              dict(label = "Integrator", values = dataset["Integrator"],
                                   tickvals = [0,1,2,3],
                                   ticktext = ["RK4", "CF4", "EE", "LE"]),
                              dict(label = 'log10(E1)', values = (dataset['E1']), tickvals = vecE1 ),
                              dict(label = 'log10(E2)', values = (dataset['E2']), tickvals = vecE2),
                              dict(label = 'log10(Loss)', values = (dataset['Training Loss']), tickvals = vecLoss)
                          ])
                          )
                              )
fig = fig.update_layout(
    title=dict(text = "Medians over the 5 repeated experiments", x=.5, xanchor="center", yanchor= "top")
)
fig = fig.update_traces(tickfont_size=15, labelfont_size = 15, 
                    tickfont_color = "black", selector=dict(type='parcoords'),
                    labelside = "bottom")
#fig.show() # this is the plot downloaded as html and displayed as interactive plot below
url = "https://htmlpreview.github.io/?https://github.com/davidemurari/learningConstrainedHamiltonians/blob/main/Constrained%20Systems/MedianExperiments.html"
IFrame(src=url, width=1000, height=500)

## Impact of $N$ and $M$ with and without noise

### Case $\varepsilon = 0$, i.e. no noise in the training trajectories

In [10]:
dataset = pd.read_csv('CollectedData.csv')
dataset = dataset.drop("Unnamed: 0",axis=1)

In [11]:
dfOrd4 = dataset[(dataset["Integrator"]<2) & (dataset["Epsilon"]==0)].groupby(['N',"M","Integrator"],as_index=False)["E1"].apply(gmean)
dfOrd4["E2"] = dataset[(dataset["Integrator"]<2) & (dataset["Epsilon"]==0)].groupby(['N',"M","Integrator"],as_index=False)["E2"].apply(gmean)["E2"]
dfOrd4["Training Loss"] = dataset[(dataset["Integrator"]<2) & (dataset["Epsilon"]==0)].groupby(['N',"M","Integrator"],as_index=False)["Training Loss"].apply(gmean)["Training Loss"]

dfOrd4 = dfOrd4.sort_values("E1")

dfOrd1 = dataset[(dataset["Integrator"]>1) & (dataset["Epsilon"]==0)].groupby(['N',"M","Integrator"],as_index=False)["E1"].apply(gmean)
dfOrd1["E2"] = dataset[(dataset["Integrator"]>1) & (dataset["Epsilon"]==0)].groupby(['N',"M","Integrator"],as_index=False)["E2"].apply(gmean)["E2"]
dfOrd1["Training Loss"] = dataset[(dataset["Integrator"]>1) & (dataset["Epsilon"]==0)].groupby(['N',"M","Integrator"],as_index=False)["Training Loss"].apply(gmean)["Training Loss"]

dfOrd1 = dfOrd1.sort_values("E1")

In [12]:
listIntegrators = ["RK4", "CF4", "EE", "LE"]
dfOrd1["Integrator"] = dfOrd1["Integrator"].astype(np.compat.long)
dfOrd4["Integrator"] = dfOrd4["Integrator"].astype(np.compat.long)

dfOrd1["Integrator"] = [listIntegrators[i] for i in  dfOrd1["Integrator"]]
dfOrd4["Integrator"] = [listIntegrators[i] for i in  dfOrd4["Integrator"]]

Combinations giving best $\mathcal{E}_{1}$ values, with methods of order 1

In [13]:
dfOrd1.head()

,N,M,Integrator,E1,E2,Training Loss
23,1500.0,5.0,LE,0.000001,0.002385,6.225047e-09
17,1000.0,5.0,LE,0.000001,0.002330,6.351126e-09
16,1000.0,5.0,EE,0.000001,0.002355,2.500104e-08
22,1500.0,5.0,EE,0.000001,0.002521,2.904461e-08
11,500.0,5.0,LE,0.000002,0.002611,8.477618e-09


Combinations giving best $\mathcal{E}_{1}$ values, with methods of order 4

In [14]:
dfOrd4.head()

,N,M,Integrator,E1,E2,Training Loss
19,1500.0,2.0,CF4,3.181553e-08,0.000134,5.457524e-11
22,1500.0,5.0,RK4,3.257486e-08,0.000140,2.036351e-11
20,1500.0,3.0,RK4,3.442145e-08,0.000144,2.420678e-11
18,1500.0,2.0,RK4,3.586097e-08,0.000149,5.758179e-11
21,1500.0,3.0,CF4,3.713637e-08,0.000145,2.887385e-11


### Case $\varepsilon>0$

dataset = pd.read_csv('CollectedData.csv')
dataset = dataset.drop("Unnamed: 0",axis=1)

In [15]:
dfOrd4 = dataset[(dataset["Integrator"]<2) & (dataset["Epsilon"]>0)].groupby(['N',"M","Integrator"],as_index=False)["E1"].apply(gmean)
dfOrd4["E2"] = dataset[(dataset["Integrator"]<2) & (dataset["Epsilon"]>0)].groupby(['N',"M","Integrator"],as_index=False)["E2"].apply(gmean)["E2"]
dfOrd4["Training Loss"] = dataset[(dataset["Integrator"]<2) & (dataset["Epsilon"]>0)].groupby(['N',"M","Integrator"],as_index=False)["Training Loss"].apply(gmean)["Training Loss"]

dfOrd4 = dfOrd4.sort_values("E1")

dfOrd1 = dataset[(dataset["Integrator"]>1) & (dataset["Epsilon"]>0)].groupby(['N',"M","Integrator"],as_index=False)["E1"].apply(gmean)
dfOrd1["E2"] = dataset[(dataset["Integrator"]>1) & (dataset["Epsilon"]>0)].groupby(['N',"M","Integrator"],as_index=False)["E2"].apply(gmean)["E2"]
dfOrd1["Training Loss"] = dataset[(dataset["Integrator"]>1) & (dataset["Epsilon"]>0)].groupby(['N',"M","Integrator"],as_index=False)["Training Loss"].apply(gmean)["Training Loss"]

dfOrd1 = dfOrd1.sort_values("E1")

In [16]:
listIntegrators = ["RK4", "CF4", "EE", "LE"]
dfOrd1["Integrator"] = dfOrd1["Integrator"].astype(np.compat.long)
dfOrd4["Integrator"] = dfOrd4["Integrator"].astype(np.compat.long)

dfOrd1["Integrator"] = [listIntegrators[i] for i in  dfOrd1["Integrator"]]
dfOrd4["Integrator"] = [listIntegrators[i] for i in  dfOrd4["Integrator"]]

Combinations giving best $\mathcal{E}_{1}$ values, with methods of order 1

In [17]:
dfOrd1.head()

,N,M,Integrator,E1,E2,Training Loss
23,1500.0,5.0,LE,0.000012,0.006607,0.000002
22,1500.0,5.0,EE,0.000012,0.006327,0.000002
17,1000.0,5.0,LE,0.000015,0.006606,0.000002
16,1000.0,5.0,EE,0.000016,0.006805,0.000002
11,500.0,5.0,LE,0.000018,0.006698,0.000002


Combinations giving best $\mathcal{E}_{1}$ values, with methods of order 4

In [18]:
dfOrd4.head()

,N,M,Integrator,E1,E2,Training Loss
22,1500.0,5.0,RK4,0.000005,0.003361,0.000002
23,1500.0,5.0,CF4,0.000006,0.004126,0.000002
17,1000.0,5.0,CF4,0.000007,0.003840,0.000002
16,1000.0,5.0,RK4,0.000008,0.004317,0.000002
14,1000.0,3.0,RK4,0.000008,0.004237,0.000002
